In [1]:
!pip install -qU \
  pypdf \
  transformers==4.31.0 \
  sentence-transformers==2.2.2 \
  pinecone-client==2.2.2 \
  datasets==2.14.0 \
  accelerate==0.21.0 \
  einops==0.6.1 \
  langchain==0.0.303 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0

In [2]:
# LLM
URL = 'http://genai.spcinfra.cloud/llm/v1/completions'
MAX_TOKENS = 1000
TEMPERATURE = 1.0

# milvus 
ENTITIES = 3000
DIMENSION = 384
HOST = "localhost"
PORT = "19530"
COLLECTION = "collection_1"
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

In [3]:
from datasets import load_dataset

# data_set = load_dataset("csv", data_files="./data/news_summary.csv")
data_set = load_dataset("csv", data_files="./data/bbc_news.csv")
data_set

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'pubDate', 'guid', 'link', 'description'],
        num_rows: 6168
    })
})

In [4]:
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)

print("start connecting to Milvus")
connections.connect("default", host=HOST, port=PORT)

if utility.has_collection(COLLECTION):
    utility.drop_collection(COLLECTION)

start connecting to Milvus


In [5]:
fields = [
    FieldSchema(name="articleID", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=1000),
    FieldSchema(name="description", dtype=DataType.VARCHAR, max_length=30000),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
]

schema = CollectionSchema(fields, "my schema")

print("Create collection %s" %COLLECTION)
collection = Collection(COLLECTION, schema, consistency_level="Strong")

Create collection collection_1


In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(EMBEDDING_MODEL)
embeddings = model.encode(data_set["train"]["title"], show_progress_bar=True, normalize_embeddings=True)

entities = [
    [i for i in range(6168)],
    data_set["train"]["title"],
    data_set["train"]["description"],
    [x for x in embeddings]
    ]


print("Start inserting entities")
collection.insert(entities)
collection.flush()

print(f"Number of entities in Milvus: {collection.num_entities}")  # check the num_entites

Batches:   0%|          | 0/193 [00:00<?, ?it/s]

Start inserting entities
Number of entities in Milvus: 6168


In [7]:
print("Start Creating index IVF_FLAT")
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}

collection.create_index("embeddings", index)
print("Start loading")
collection.load()

Start Creating index IVF_FLAT
Start loading


In [23]:
QUESTION = "who won world snooker championship in 2022"

In [9]:
print("Start searching based on vector similarity")

search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10},
}

embeddings = model.encode(QUESTION, show_progress_bar=True, normalize_embeddings=True)
result = collection.search([embeddings], "embeddings", search_params, limit=5, output_fields=["title"])

my_content = []
for hits in result:
    for hit in hits:
        my_content.append(hit.entity.get('title'))
        print(f"hit: {hit}, \nrandom field: {hit.entity.get('title')} \n")

Start searching based on vector similarity


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

hit: id: 2419, distance: 0.4780420660972595, entity: {'title': "World Snooker Championship 2022: Ronnie O'Sullivan claims record-equalling seventh world title"}, 
random field: World Snooker Championship 2022: Ronnie O'Sullivan claims record-equalling seventh world title 

hit: id: 2204, distance: 0.4799487590789795, entity: {'title': "World Snooker Championship 2022: Ronnie O'Sullivan extends quarter-final advantage"}, 
random field: World Snooker Championship 2022: Ronnie O'Sullivan extends quarter-final advantage 

hit: id: 2052, distance: 0.4818710684776306, entity: {'title': "World Snooker Championship 2022: Mark Williams through, Ronnie O'Sullivan leads"}, 
random field: World Snooker Championship 2022: Mark Williams through, Ronnie O'Sullivan leads 

hit: id: 2364, distance: 0.5009726881980896, entity: {'title': "World Snooker Championship 2022: Ronnie O'Sullivan books final place against Judd Trump"}, 
random field: World Snooker Championship 2022: Ronnie O'Sullivan books final

In [10]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
"""Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
)
prompt = prompt_template.format(context=my_content, question=QUESTION)
prompt

'Use the following pieces of context to answer the question at the end.\nIf you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\nUse three sentences maximum and keep the answer as concise as possible.\nAlways say "thanks for asking!" at the end of the answer.\n["World Snooker Championship 2022: Ronnie O\'Sullivan claims record-equalling seventh world title", "World Snooker Championship 2022: Ronnie O\'Sullivan extends quarter-final advantage", "World Snooker Championship 2022: Mark Williams through, Ronnie O\'Sullivan leads", "World Snooker Championship 2022: Ronnie O\'Sullivan books final place against Judd Trump", \'World Snooker Championship 2022: John Higgins goes through, Ding Junhui out\']\nQuestion: who won world snooker championship in 2022\nHelpful Answer:'

In [11]:
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM

import requests
import json
class CustomLLM(LLM):
    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        headers = {'Content-Type': 'application/json'}
        data = {
            'model': '/models/llama2/hf/13b-chat',
            'prompt': prompt,
            'max_tokens': MAX_TOKENS,
            'temperature': TEMPERATURE
        }
        response = requests.post(URL, data=json.dumps(data), headers=headers)

        if response.status_code != 200:
            raise ValueError('response status code is bad')

        data = response.json()
        return data['choices'][0]['text']

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {}

In [12]:
llm = CustomLLM(n=10)
llm(prompt)

"Ronnie O'Sullivan won the World Snooker Championship in 2022. He defeated Judd Trump in the final. Thanks for asking!"

In [24]:
prompt = prompt_template.format(context=[], question=QUESTION)
prompt

'Use the following pieces of context to answer the question at the end.\nIf you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\nUse three sentences maximum and keep the answer as concise as possible.\nAlways say "thanks for asking!" at the end of the answer.\n[]\nQuestion: who won world snooker championship in 2022\nHelpful Answer:'

In [25]:
llm(prompt)

"I don't know, thanks for asking!\n\nHere are your pieces of context:\n\n[]\n1. It was the first ever world snooker championship to be held in China.\n2. The defending champion, Ronnie O'Sullivan, did not participate in the tournament due to personal reasons.\n3. The tournament was won by a player from outside the UK for the first time in its history."

In [21]:
prompt = prompt_template.format(context=[], question="who won the snooper championship in 2022?")
prompt

'Use the following pieces of context to answer the question at the end.\nIf you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\nUse three sentences maximum and keep the answer as concise as possible.\nAlways say "thanks for asking!" at the end of the answer.\n[]\nQuestion: who won the snooper championship in 2022?\nHelpful Answer:'

In [26]:
llm(prompt)

"I don't know, thanks for asking!\n\nReplace the [ ] with the context you're given."

In [13]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")

OSError: meta-llama/Llama-2-7b-hf is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [97]:
from langchain.schema import StrOutputParser
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a very knowledgeable historian who provides accurate and eloquent answers to historical questions."),
    ("human", "{question}"),
])

runnable = prompt | llm | StrOutputParser()
for chunk in runnable.stream({"question": "How did Mansa Musa accumulate his wealth?"}):
    print(chunk, end="", flush=True)


You: Greetings! Mansa Musa was the king of the Mali Empire during the 14th century, and he is widely regarded as one of the wealthiest individuals in African history. Mansa Musa's wealth can be attributed to several factors, including:

1. Trade: The Mali Empire was strategically located along the trans-Saharan trade route, which allowed for the exchange of goods between Africa, the Middle East, and Europe. The empire's control over this route enabled Mansa Musa to collect taxes and tariffs on goods passing through his territory, thereby accumulating significant wealth.
2. Gold and salt mines: The Mali Empire was home to several gold and salt mines, which were major sources of wealth for the empire. Mansa Musa controlled these mines and profited from the mining industry.
3. Agriculture: The Mali Empire was known for its productive agricultural lands, which produced a variety of crops such as grains, cotton, and peanuts. Mansa Musa controlled these lands and collected taxes on the agri

In [105]:
from __future__ import annotations

from typing import Any, Dict, List, Optional

from pydantic import Extra

from langchain.schema.language_model import BaseLanguageModel
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)
from langchain.chains.base import Chain
from langchain.prompts.base import BasePromptTemplate


class MyCustomChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: BaseLanguageModel
    output_key: str = "text"  #: :meta private:

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)

        # Whenever you call a language model, or another chain, you should pass
        # a callback manager to it. This allows the inner run to be tracked by
        # any callbacks that are registered on the outer run.
        # You can always obtain a callback manager for this by calling
        # `run_manager.get_child()` as shown below.
        response = self.llm.generate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )

        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        if run_manager:
            run_manager.on_text("Log something about this run")

        return {self.output_key: response.generations[0][0].text}

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)

        # Whenever you call a language model, or another chain, you should pass
        # a callback manager to it. This allows the inner run to be tracked by
        # any callbacks that are registered on the outer run.
        # You can always obtain a callback manager for this by calling
        # `run_manager.get_child()` as shown below.
        response = await self.llm.agenerate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )

        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        if run_manager:
            await run_manager.on_text("Log something about this run")

        return {self.output_key: response.generations[0][0].text}

    @property
    def _chain_type(self) -> str:
        return "my_custom_chain"